In [1]:
import os
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [2]:
# Specify the directory path
root = '/gdrive/MyDrive/Irfan/ESC-50-master'
!ls '/gdrive/MyDrive/Irfan/ESC-50-master'

audio			     melspectrograms	    tests
DenseNet201.best.hdf5	     meta		    VGG16.best.hdf5
esc50.gif		     pytest.ini		    VGG19.best.hdf5
InceptionResNetV2.best.hdf5  README.md		    Xception.best.hdf5
InceptionV3.best.hdf5	     requirements.txt
LICENSE			     ResNet152V2.best.hdf5


In [21]:
from sklearn.metrics import accuracy_score
from keras.models import load_model
from keras.utils import to_categorical
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers.merge import concatenate
from numpy import argmax

from keras import metrics, optimizers
from keras.preprocessing import image
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Input
from keras.applications.vgg16 import preprocess_input
from matplotlib import pyplot as plt
import numpy as np
import json as js
import tensorflow as tf

from sklearn.linear_model import LogisticRegression
from keras.models import load_model
from keras.utils import to_categorical
from numpy import dstack

In [4]:
batch_size = 32
epochs = 100

# dimensions of our images.
img_width, img_height = 224, 224

input_tensor = Input(shape=(224,224,3))

nb_training_samples = 1600
nb_validation_samples = 400# Set parameter values

In [5]:
# training generator configuration
training_data_dir = root + '/melspectrograms/training'

training_datagen = image.ImageDataGenerator(
    rescale=1./255)

training_generator = training_datagen.flow_from_directory(
    training_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size)

# validation generator configuration
validation_data_dir = root + '/melspectrograms/testing/'

validation_datagen = image.ImageDataGenerator(
    rescale=1./255)

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size)

trainX, trainy = next(training_generator)
testX, testy = next(validation_generator)

Found 1600 images belonging to 50 classes.
Found 400 images belonging to 50 classes.


In [6]:
def top_5_accuracy(y_true, y_pred):
      return metrics.top_k_categorical_accuracy(y_true, y_pred, k=5)

In [7]:
def load_all_models(models_name):
  # Specify the directory path
  load_root = '/gdrive/MyDrive/Irfan/models/'
  all_models = []
  for i in range(5):
    # define filename for this ensemble
    filename = load_root + models_name[i] + '.h5'
		# load model from file
    model = load_model(filename, custom_objects={'top_5_accuracy': top_5_accuracy})
    # add to list of members
    all_models.append(model)
    print('>loaded %s' % filename)
  return all_models

In [8]:
# create stacked model input dataset as outputs from the ensemble
def stacked_dataset(members, inputX):
	stackX = None
	for model in members:
		# make prediction
		yhat = model.predict(inputX, verbose=0)
		# stack predictions into [rows, members, probabilities]
		if stackX is None:
			stackX = yhat
		else:
			stackX = dstack((stackX, yhat))
	# flatten predictions to [rows, members x probabilities]
	stackX = stackX.reshape((stackX.shape[0], stackX.shape[1]*stackX.shape[2]))
	return stackX

In [9]:
# fit a model based on the outputs from the ensemble members
def fit_stacked_model(members, inputX, inputy):
	# create dataset using ensemble
	stackedX = stacked_dataset(members, inputX)
	# fit standalone model
	model = LogisticRegression()
	model.fit(stackedX, inputy)
	return model

In [10]:
# make a prediction with the stacked model
def stacked_prediction(members, model, inputX):
	# create dataset using ensemble
	stackedX = stacked_dataset(members, inputX)
	# make a prediction
	yhat = model.predict(stackedX)
	return yhat

In [19]:
models_name = ['DenseNet201', 'VGG19', 'ResNet152V2', 'InceptionV3', 'Xception']
members = load_all_models(models_name)

>loaded /gdrive/MyDrive/Irfan/models/DenseNet201.h5
>loaded /gdrive/MyDrive/Irfan/models/VGG19.h5
>loaded /gdrive/MyDrive/Irfan/models/ResNet152V2.h5
>loaded /gdrive/MyDrive/Irfan/models/InceptionV3.h5
>loaded /gdrive/MyDrive/Irfan/models/Xception.h5


In [22]:
# fit stacked model using the ensemble
model = fit_stacked_model(members, testX, testy)
# evaluate model on test set
yhat = stacked_prediction(members, model, testX)
acc = accuracy_score(testy, yhat)
print('Stacked Test Accuracy: %.3f' % acc)

ValueError: ignored